In [1]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

2024-03-25 22:57:03.221262: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 22:57:03.251580: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 22:57:03.252903: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 22:57:03.884828: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data=pd.read_csv('final_english_dataset_with_preprocess_on_overview.csv')

In [3]:
data.head()

,budget,genres,id,original_language,original_title,overview,popularity,production_countries,release_date,revenue,...,director,screenplay,producer,preprocess_overviews,preprocess_overviews_word2vec,top_actor,director_preprocess,producer_preprocess,genres_preprocess,new_column
0,30000000.0,"['Animation', 'Comedy', 'Family']",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,United States of America,1995-10-30,373554033.0,...,['John Lasseter'],"['Joss Whedon', 'Andrew Stanton', 'Joel Cohen'...","['Bonnie Arnold', 'Ralph Guggenheim']","led woody , andy 's toy live happily room andy...","['led', 'woody', 'andys', 'toy', 'live', 'happ...","tomhanks, timallen, donrickles, jimvarney, wal...",johnlasseter,"bonniearnold, ralphguggenheim","animation, comedy, family","animation, comedy, family johnlasseter tomhank..."
1,65000000.0,"['Adventure', 'Fantasy', 'Family']",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,United States of America,1995-12-15,262797249.0,...,['Joe Johnston'],"['Jonathan Hensleigh', 'Greg Taylor', 'Jim Str...","['Scott Kroopf', 'William Teitler']",sibling judy peter discover enchanted board ga...,"['sibling', 'judy', 'peter', 'discover', 'ench...","robinwilliams, jonathanhyde, kirstendunst, bra...",joejohnston,"scottkroopf, williamteitler","adventure, fantasy, family","adventure, fantasy, family joejohnston robinwi..."
2,0.0,"['Romance', 'Comedy']",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,United States of America,1995-12-22,0.0,...,['Howard Deutch'],[],[],family wedding reignites ancient feud next-doo...,"['family', 'wedding', 'reignites', 'ancient', ...","waltermatthau, jacklemmon, ann-margret, sophia...",howarddeutch,NaN,"romance, comedy","romance, comedy howarddeutch waltermatthau, ja..."
3,16000000.0,"['Comedy', 'Drama', 'Romance']",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,United States of America,1995-12-22,81452156.0,...,['Forest Whitaker'],"['Ronald Bass', 'Terry McMillan']","['Ronald Bass', 'Ezra Swerdlow', 'Deborah Schi...","cheated , mistreated stepped , woman holding b...","['cheated', 'mistreated', 'stepped', 'woman', ...","whitneyhouston, angelabassett, lorettadevine, ...",forestwhitaker,"ronaldbass, ezraswerdlow, deborahschindler, ca...","comedy, drama, romance","comedy, drama, romance forestwhitaker whitneyh..."
4,0.0,['Comedy'],11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,United States of America,1995-02-10,76578911.0,...,['Charles Shyer'],"['Nancy Meyers', 'Albert Hackett']",['Nancy Meyers'],"george bank recovered daughter 's wedding , re...","['george', 'bank', 'recovered', 'daughter', 'w...","stevemartin, dianekeaton, martinshort, kimberl...",charlesshyer,nancymeyers,comedy,"comedy charlesshyer stevemartin, dianekeaton, ..."


In [4]:
data.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,2.430200e+04,24302.000000,24302.000000,2.430200e+04,24302.000000,24302.000000,24302.000000
mean,5.456504e+06,100172.573615,3.439987,1.395673e+07,94.663773,5.501975,139.617357
std,1.881566e+07,107986.359996,7.041831,6.740897e+07,36.331035,1.877977,558.922008
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,23049.250000,0.444847,0.000000e+00,86.000000,5.000000,3.000000
50%,0.000000e+00,51993.500000,1.329848,0.000000e+00,94.000000,5.900000,11.000000
75%,0.000000e+00,139150.750000,5.069707,0.000000e+00,105.000000,6.600000,45.000000
max,2.700000e+08,467731.000000,547.488298,2.787965e+09,1256.000000,10.000000,14075.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24302 entries, 0 to 24301
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   budget                         24302 non-null  float64
 1   genres                         24302 non-null  object 
 2   id                             24302 non-null  int64  
 3   original_language              24302 non-null  object 
 4   original_title                 24302 non-null  object 
 5   overview                       24302 non-null  object 
 6   popularity                     24302 non-null  float64
 7   production_countries           20658 non-null  object 
 8   release_date                   24302 non-null  object 
 9   revenue                        24302 non-null  float64
 10  runtime                        24302 non-null  float64
 11  spoken_languages               21942 non-null  object 
 12  tagline                        14251 non-null 

In [6]:
#overview, tagline, keywords, preprocess_overviews, preprocess_overviews_word2vec

In [7]:
data.loc[:,['overview', 'tagline', 'keywords', 'preprocess_overviews', 'preprocess_overviews_word2vec']].head()

,overview,tagline,keywords,preprocess_overviews,preprocess_overviews_word2vec
0,"Led by Woody, Andy's toys live happily in his ...",NaN,"['jealousy', 'toy', 'boy', 'friendship', 'frie...","led woody , andy 's toy live happily room andy...","['led', 'woody', 'andys', 'toy', 'live', 'happ..."
1,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,"['board game', 'disappearance', ""based on chil...",sibling judy peter discover enchanted board ga...,"['sibling', 'judy', 'peter', 'discover', 'ench..."
2,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,"['fishing', 'best friend', 'duringcreditssting...",family wedding reignites ancient feud next-doo...,"['family', 'wedding', 'reignites', 'ancient', ..."
3,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,"['based on novel', 'interracial relationship',...","cheated , mistreated stepped , woman holding b...","['cheated', 'mistreated', 'stepped', 'woman', ..."
4,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,"['baby', 'midlife crisis', 'confidence', 'agin...","george bank recovered daughter 's wedding , re...","['george', 'bank', 'recovered', 'daughter', 'w..."


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Création du vecteur TF-IDF pour les résumés des films
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['overview'].fillna(''))

# Fonction de recommandation
def recommend_movies(input_description, n=10):
    # Encodage de la description de l'utilisateur
    input_vector = tfidf_vectorizer.transform([input_description])

    # Calcul de la similarité cosinus entre la description de l'utilisateur et les résumés des films
    cosine_similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()

    # Trie les films par similarité et récupère les indices des films recommandés
    recommended_movie_indices = cosine_similarities.argsort()[:-n-1:-1]

    # Récupère les détails des films recommandés
    recommended_movies = data.iloc[recommended_movie_indices]

    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]

# Exemple d'utilisation
user_input = "A movie like the batman series. Something dark but fun with heroes"
recommendations = recommend_movies(user_input)
display(recommendations)


,original_title,genres,release_date,overview
965,Batman Returns,"['Action', 'Fantasy']",1992-06-19,"Having defeated the Joker, Batman now faces th..."
22956,The Lego Batman Movie,"['Action', 'Animation', 'Comedy', 'Family', 'F...",2017-02-08,In the irreverent spirit of fun that made “The...
3308,Liebestraum,"['Horror', 'Thriller', 'Mystery', 'Romance']",1991-09-13,A man returns to his hometown and a series of ...
2416,Born American,"['Drama', 'Action']",1986-08-30,Three American students vacationing in Finland...
11370,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']",2012-07-16,Following the death of District Attorney Harve...
112,Batman Forever,"['Action', 'Crime', 'Fantasy']",1995-06-16,The Dark Knight of Gotham City confronts a das...
8255,The Dark Knight,"['Drama', 'Action', 'Crime', 'Thriller']",2008-07-16,Batman raises the stakes in his war on crime. ...
19871,Starring Adam West,['Documentary'],2013-03-13,A documentary following the career ups and dow...
11684,Midnight Movie,['Horror'],2008-10-24,"In the 1970's, a slasher flick was released na..."
8508,Disaster Movie,"['Action', 'Comedy']",2008-08-29,"In DISASTER MOVIE, the filmmaking team behind ..."


In [9]:
data.head()

,budget,genres,id,original_language,original_title,overview,popularity,production_countries,release_date,revenue,...,director,screenplay,producer,preprocess_overviews,preprocess_overviews_word2vec,top_actor,director_preprocess,producer_preprocess,genres_preprocess,new_column
0,30000000.0,"['Animation', 'Comedy', 'Family']",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,United States of America,1995-10-30,373554033.0,...,['John Lasseter'],"['Joss Whedon', 'Andrew Stanton', 'Joel Cohen'...","['Bonnie Arnold', 'Ralph Guggenheim']","led woody , andy 's toy live happily room andy...","['led', 'woody', 'andys', 'toy', 'live', 'happ...","tomhanks, timallen, donrickles, jimvarney, wal...",johnlasseter,"bonniearnold, ralphguggenheim","animation, comedy, family","animation, comedy, family johnlasseter tomhank..."
1,65000000.0,"['Adventure', 'Fantasy', 'Family']",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,United States of America,1995-12-15,262797249.0,...,['Joe Johnston'],"['Jonathan Hensleigh', 'Greg Taylor', 'Jim Str...","['Scott Kroopf', 'William Teitler']",sibling judy peter discover enchanted board ga...,"['sibling', 'judy', 'peter', 'discover', 'ench...","robinwilliams, jonathanhyde, kirstendunst, bra...",joejohnston,"scottkroopf, williamteitler","adventure, fantasy, family","adventure, fantasy, family joejohnston robinwi..."
2,0.0,"['Romance', 'Comedy']",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,United States of America,1995-12-22,0.0,...,['Howard Deutch'],[],[],family wedding reignites ancient feud next-doo...,"['family', 'wedding', 'reignites', 'ancient', ...","waltermatthau, jacklemmon, ann-margret, sophia...",howarddeutch,NaN,"romance, comedy","romance, comedy howarddeutch waltermatthau, ja..."
3,16000000.0,"['Comedy', 'Drama', 'Romance']",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,United States of America,1995-12-22,81452156.0,...,['Forest Whitaker'],"['Ronald Bass', 'Terry McMillan']","['Ronald Bass', 'Ezra Swerdlow', 'Deborah Schi...","cheated , mistreated stepped , woman holding b...","['cheated', 'mistreated', 'stepped', 'woman', ...","whitneyhouston, angelabassett, lorettadevine, ...",forestwhitaker,"ronaldbass, ezraswerdlow, deborahschindler, ca...","comedy, drama, romance","comedy, drama, romance forestwhitaker whitneyh..."
4,0.0,['Comedy'],11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,United States of America,1995-02-10,76578911.0,...,['Charles Shyer'],"['Nancy Meyers', 'Albert Hackett']",['Nancy Meyers'],"george bank recovered daughter 's wedding , re...","['george', 'bank', 'recovered', 'daughter', 'w...","stevemartin, dianekeaton, martinshort, kimberl...",charlesshyer,nancymeyers,comedy,"comedy charlesshyer stevemartin, dianekeaton, ..."


In [10]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# Entraîner un modèle Word2Vec sur les données d'overview
sentences = [overview.split() for overview in data['overview'].fillna("")]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)

def recommend_movies_word2vec(input_description, n=10):
    input_vector = np.mean([word2vec_model.wv[word] for word in input_description.split() if word in word2vec_model.wv], axis=0)
    if np.isnan(input_vector).any() or input_vector.shape == ():
        return "No valid input vector"
    
    similarity_scores = []
    for overview in data['overview']:
        overview_vector = np.mean([word2vec_model.wv[word] for word in overview.split() if word in word2vec_model.wv], axis=0)
        if np.isnan(overview_vector).any() or overview_vector.shape == ():
            similarity_scores.append(0)  # Adding 0 for invalid overview vectors
        else:
            similarity_scores.append(cosine_similarity([input_vector], [overview_vector])[0][0])
    
    recommended_movie_indices = np.argsort(similarity_scores)[-n:][::-1]
    recommended_movies = data.iloc[recommended_movie_indices]
    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]

# Exemple d'utilisation
user_input = "A movie like the batman series. Something dark but fun with heroes"
recommendations = recommend_movies_word2vec(user_input)
display(recommendations)



,original_title,genres,release_date,overview
8616,The Living Sea,['Documentary'],1995-02-01,"The Living Sea, a film about the world's ocean..."
13886,Meth,[],2006-01-01,But it is a devastating expose of party-n-play...
15517,En som deg,['Romance'],2012-10-18,A Finnish-Norwegian love story with an unexpec...
16655,Tig,['Documentary'],2015-04-23,"An intimate, mixed media documentary that foll..."
9384,Objectified,['Documentary'],2009-03-14,A feature-length documentary about our complex...
13961,Scary or Die,['Horror'],2012-09-09,"The creation of filmmaker Michael Emanuel, SCA..."
24298,Robin Hood,"['Drama', 'Action', 'Romance']",1991-05-13,"Yet another version of the classic epic, with ..."
17511,Pelota,[],1983-09-09,A documentary view of the Basque ball-game in ...
20800,The Plank,['Comedy'],1967-06-09,"Classic short British comedy, full of stars, a..."
20998,Le Chant Des Ondes,['Documentary'],2013-07-06,WAVEMAKERS is a feature-length documentary tra...


In [11]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

# Créer des documents tagués pour Doc2Vec
tagged_documents = [TaggedDocument(words=overview.split(), tags=[str(i)]) for i, overview in enumerate(data['overview'].fillna(""))]

# Entraîner un modèle Doc2Vec
doc2vec_model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, epochs=20)

# Fonction de recommandation utilisant Doc2Vec
def recommend_movies_doc2vec(input_description, n=10):
    input_vector = doc2vec_model.infer_vector(input_description.split())
    similarity_scores = []
    for i, overview in enumerate(data['overview']):
        overview_vector = doc2vec_model.docvecs[str(i)]
        similarity_scores.append(cosine_similarity([input_vector], [overview_vector])[0][0])
    recommended_movie_indices = np.argsort(similarity_scores)[-n:][::-1]
    recommended_movies = data.iloc[recommended_movie_indices]
    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]

# Exemple d'utilisation
user_input = "A movie like the batman series. Something dark but fun with heroes"
recommendations = recommend_movies_doc2vec(user_input)
display(recommendations)

,original_title,genres,release_date,overview
9893,Marie and Bruce,"['Comedy', 'Romance']",2004-01-19,"Based on a play by Wallace Shawn, the film is ..."
10888,See This Movie,"['Comedy', 'Drama']",2004-03-03,Two bumbling but determined three-day film sch...
13886,Meth,[],2006-01-01,But it is a devastating expose of party-n-play...
22620,Lunch Wagon,"['Comedy', 'Crime']",1981-09-01,Three women start a lunch wagon business but r...
16395,Hot Saturday,['Drama'],1932-10-28,A pretty but virtuous small-town bank clerk is...
19589,The Challenge,"['Action', 'Adventure', 'Comedy', 'Family', 'S...",2003-01-24,The Challenge is an action adventure/adventure...
13301,A Trip to Mars,"['Fantasy', 'Adventure']",1910-02-18,The obvious inspiration for this short film is...
20614,The Ghost Train,['Documentary'],1901-01-21,Special effects film with a train double expos...
19426,The Dead Zone,"['Drama', 'Science Fiction', 'Thriller', 'TV M...",2002-11-05,A young man awakens from a six-year coma with ...
1203,Afterglow,"['Drama', 'Romance', 'Comedy']",1997-12-26,A handyman with marital problems meets a house...


In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Charger un modèle BERT pré-entraîné
bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

# Fonction de recommandation utilisant BERT
def recommend_movies_bert(input_description, n=10):
    input_embedding = bert_model.encode(input_description)
    overview_embeddings = bert_model.encode(data['overview'].fillna("").tolist())
    similarity_scores = cosine_similarity([input_embedding], overview_embeddings)[0]
    recommended_movie_indices = np.argsort(similarity_scores)[-n:][::-1]
    recommended_movies = data.iloc[recommended_movie_indices]
    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]

# Exemple d'utilisation
user_input = "A movie like the batman series. Something dark but fun with heroes"
recommendations = recommend_movies_bert(user_input)
display(recommendations)

,original_title,genres,release_date,overview
22956,The Lego Batman Movie,"['Action', 'Animation', 'Comedy', 'Family', 'F...",2017-02-08,In the irreverent spirit of fun that made “The...
21335,Angel City,[],1977-10-18,A detective fiction mixed with an essay-docume...
21860,Ciuleandra,"['Drama', 'Romance']",1985-01-14,The Director managed to captures perfectly the...
24298,Robin Hood,"['Drama', 'Action', 'Romance']",1991-05-13,"Yet another version of the classic epic, with ..."
18027,My Big Fat Independent Movie,['Comedy'],2005-03-04,"This film is a spoof along the lines of ""Scary..."
4901,A Shot in the Dark,"['Comedy', 'Crime']",1964-06-23,A Shot in the Dark is the second and more succ...
19646,Baji,['Action'],2015-02-06,It is an action-adventure-romance laced superh...
16499,Within the Woods,['Horror'],1978-11-23,The low budget film starring the young Bruce C...
12306,Campfire Tales,['Horror'],1997-05-16,"Anthology of famous, scary urban legends done ..."
20437,Amerikana,[],2001-05-15,Following the Dogma rules does not make this f...


In [13]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Créer une matrice TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data['overview'].fillna(''))

# Réduire la dimensionnalité avec LSA
lsa_model = TruncatedSVD(n_components=100)
lsa_matrix = lsa_model.fit_transform(tfidf_matrix)

# Fonction de recommandation utilisant LSA
def recommend_movies_lsa(input_description, n=10):
    input_vector = tfidf_vectorizer.transform([input_description])
    input_lsa = lsa_model.transform(input_vector)
    similarity_scores = cosine_similarity(input_lsa, lsa_matrix).flatten()
    recommended_movie_indices = np.argsort(similarity_scores)[-n:][::-1]
    recommended_movies = data.iloc[recommended_movie_indices]
    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]

# Exemple d'utilisation
user_input = "A movie like the batman series. Something dark but fun with heroes"
recommendations = recommend_movies_lsa(user_input)
display(recommendations)

,original_title,genres,release_date,overview
9365,Turtles Forever,"['Science Fiction', 'Adventure', 'Animation', ...",2009-11-21,Turtles Forever is a made-for-tv animated movi...
18524,Cowboy in Sweden,"['TV Movie', 'Music']",1970-01-01,Lee Hazlewood's album “Cowboy in Sweden” prese...
2646,The Kentucky Fried Movie,['Comedy'],1977-08-10,A series of loosely connected skits that spoof...
23823,The Sinister Urge,"['Crime', 'Drama', 'Thriller']",1960-12-08,A flunky for a porno movie ring starts murderi...
11933,Aliisa,[],1970-11-18,A finnish TV-movie by Jukka Sipilä
6946,Movie Movie,['Comedy'],1978-11-01,"Three movie genres of the 1930s, boxing films,..."
18091,Freaky Friday,"['TV Movie', 'Comedy', 'Family', 'Fantasy']",1995-05-06,TV remake of 1976 movie Freaky Friday.
18480,The Pirates of Penzance,"['Music', 'Comedy', 'Adventure']",1980-06-01,This Pirates of Penzance is primarily a histor...
13935,Night Gallery,[],1969-04-03,Night Gallery launched as a television movie o...
23565,Binky Nelson Unpacified,"['Family', 'Animation']",2015-11-04,"Little baby needs his pacifier, but loses it d..."


In [14]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Créer une matrice de documents-termes avec CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')
dtm = count_vectorizer.fit_transform(data['overview'].fillna(''))

# Modèle de topic modeling LDA
lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
lda_model.fit(dtm)

def recommend_movies_lda(input_description, n=10):
    input_vector = count_vectorizer.transform([input_description])
    input_topic_dist = lda_model.transform(input_vector)
    
    # Aplatir les vecteurs de distribution de thèmes LDA
    input_topic_dist = input_topic_dist.reshape(1, -1)
    dtm_topic_dist = lda_model.transform(dtm)
    
    similarity_scores = cosine_similarity(input_topic_dist, dtm_topic_dist)
    
    recommended_movie_indices = np.argsort(similarity_scores.flatten())[-n:][::-1]
    recommended_movies = data.iloc[recommended_movie_indices]
    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]

# Exemple d'utilisation
user_input = "A movie like the batman series. Something dark but fun with heroes"
recommendations = recommend_movies_lda(user_input)
display(recommendations)

,original_title,genres,release_date,overview
8172,The Cheyenne Social Club,"['Action', 'Comedy', 'Western']",1970-06-12,"Two cowboys inherit a ""social club"" specializi..."
5887,Werewolf of London,"['Fantasy', 'Horror', 'Drama']",1935-05-13,A strange animal attack turns a botanist into ...
23994,Feed the Kitty,"['Animation', 'Family', 'Comedy']",1952-02-02,"A bulldog adopts a small cat, but he can't let..."
23062,Justice League Dark,"['Action', 'Animation', 'Fantasy', 'Science Fi...",2017-01-24,Beings with supernatural powers join together ...
18990,Inkognito iz Peterburga,['Comedy'],1977-01-01,A comedy based on a classic play 'Revizor' by ...
6448,Feux rouges,"['Thriller', 'Mystery', 'Crime']",2004-03-03,A cross-country trip turns out to be a nightma...
20698,Apocalyptic,"['Horror', 'Drama']",2013-11-22,A local news crew become horribly involved wit...
19520,Extraordinary Tales,"['Horror', 'Animation', 'Mystery']",2015-10-23,An animated anthology of 5 stories adapted fro...
22211,The Legend of Nigger Charley,"['Western', 'Action', 'Crime']",1972-03-16,A trio of runaway slaves evade authorities in ...
4666,Super Sucker,['Comedy'],2002-01-01,Two door to door vaccuum cleaner salesmen hila...


In [15]:
import spacy
def recommend_movies_with_filters(input_description, n=10):
    input_vector = count_vectorizer.transform([input_description])
    input_topic_dist = lda_model.transform(input_vector)
    
    # Aplatir les vecteurs de distribution de thèmes LDA
    input_topic_dist = input_topic_dist.reshape(1, -1)
    dtm_topic_dist = lda_model.transform(dtm)
    
    # Calcul de la similarité cosinus entre le vecteur d'entrée et les vecteurs de distribution de thèmes LDA
    similarity_scores = cosine_similarity(input_topic_dist, dtm_topic_dist).flatten()
    
    # Filtrer les films basés sur la similarité cosinus
    recommended_movie_indices = np.argsort(similarity_scores)[-n:][::-1]
    recommended_movies = data.iloc[recommended_movie_indices]
    
    # Filtrer les films basés sur les genres
    if 'genres' in data.columns:
        user_genres = get_user_genres(input_description)
        if user_genres:
            recommended_movies = recommended_movies[recommended_movies['genres'].apply(lambda x: any(genre in x for genre in user_genres))]
    
    # Filtrer les films basés sur les mots-clés
    if 'keywords' in data.columns:
        user_keywords = get_user_keywords(input_description)
        if user_keywords:
            recommended_movies = recommended_movies[recommended_movies['keywords'].apply(lambda x: any(keyword in x for keyword in user_keywords))]
    
    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]

from collections import Counter

def extract_genre_keywords(data):
    genre_keywords = {}
    
    # Parcours des genres dans le dataset
    for genres in data['genres']:
        for genre in genres.split(', '):
            # Si le genre n'est pas déjà dans le dictionnaire, l'initialiser avec un compteur vide
            if genre not in genre_keywords:
                genre_keywords[genre] = Counter()
    
    # Parcours des données pour analyser les mots-clés et les synopsis pour chaque genre
    for index, row in data.iterrows():
        genres = row['genres'].split(', ')
        overview = row['overview']
        keywords = row['keywords'].split(', ')
        
        for genre in genres:
            # Ajouter les mots-clés de l'overview
            for word in overview.split():
                genre_keywords[genre][word.lower()] += 1
            
            # Ajouter les mots-clés
            for keyword in keywords:
                genre_keywords[genre][keyword.lower()] += 1
    
    # Sélectionner les 5 mots-clés les plus fréquents pour chaque genre
    for genre, counter in genre_keywords.items():
        genre_keywords[genre] = [word for word, _ in counter.most_common(5)]
    
    return genre_keywords

genre_keywords = extract_genre_keywords(data)

def get_user_keywords(input_description):
    # Vous pouvez utiliser une méthode d'extraction de mots-clés basée sur des règles pour extraire les mots-clés
    # Par exemple, vous pouvez utiliser une analyse syntaxique pour extraire les noms et les adjectifs de la description utilisateur
    
    # Placeholder pour les mots-clés extraits
    user_keywords = []
    
    # Exemple d'extraction de mots-clés en utilisant spaCy pour l'analyse syntaxique
    import spacy
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(input_description)
    
    # Extraction des noms et des adjectifs comme mots-clés
    for token in doc:
        if token.pos_ in ['NOUN', 'PROPN', 'ADJ']:
            user_keywords.append(token.text.lower())
    
    return user_keywords

def get_user_genres(input_description, genre_keywords):
    # Recherche des genres mentionnés directement dans l'input de l'utilisateur
    mentioned_genres = []
    for genre in genre_keywords.keys():
        if genre.lower() in input_description.lower():
            mentioned_genres.append(genre)
    
    # Si des genres sont mentionnés directement, les retourner
    if mentioned_genres:
        return mentioned_genres
    
    # Si aucun genre n'est mentionné directement, utiliser les mots-clés pour prédire le genre le plus probable
    input_keywords = get_user_keywords(input_description)
    predicted_genre = None
    max_overlap = 0
    
    for genre, keywords in genre_keywords.items():
        overlap = len(set(input_keywords).intersection(set(keywords)))
        if overlap > max_overlap:
            max_overlap = overlap
            predicted_genre = genre
    
    return [predicted_genre] if predicted_genre else []

# Test de la fonction get_user_genres
user_input_1 = "Je veux un film d'action avec des combats et des batailles épiques"
user_input_2 = "Un film romantique avec beaucoup de romance et d'amour"
user_input_3 = "Je cherche un thriller plein de suspense et de mystère"

print(get_user_genres(user_input_1, genre_keywords))
print(get_user_genres(user_input_2, genre_keywords))
print(get_user_genres(user_input_3, genre_keywords))


[]
[]
[]


In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def recommend_movies(input_description, embedding_method, similarity_method, n=10):
    if embedding_method == 'tfidf':
        # Création du vecteur TF-IDF pour les résumés des films
        tfidf_vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf_vectorizer.fit_transform(data['overview'].fillna(''))
        
        # Encodage de la description de l'utilisateur
        input_vector = tfidf_vectorizer.transform([input_description])
    elif embedding_method == 'word2vec':
        # Entraîner un modèle Word2Vec sur les données d'overview
        sentences = [overview.split() for overview in data['overview'].fillna("")]
        word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)
        
        input_vector = np.mean([word2vec_model.wv[word] for word in input_description.split() if word in word2vec_model.wv], axis=0)
    elif embedding_method == 'doc2vec':
        # Créer des documents tagués pour Doc2Vec
        tagged_documents = [TaggedDocument(words=overview.split(), tags=[str(i)]) for i, overview in enumerate(data['overview'].fillna(""))]
        
        # Entraîner un modèle Doc2Vec
        doc2vec_model = Doc2Vec(tagged_documents, vector_size=100, window=5, min_count=1, epochs=20)
        
        input_vector = doc2vec_model.infer_vector(input_description.split())
    elif embedding_method == 'bert':
        # Charger un modèle BERT pré-entraîné
        bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
        
        input_vector = bert_model.encode(input_description)
    elif embedding_method == 'lsa':
        # Créer une matrice TF-IDF
        tfidf_vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf_vectorizer.fit_transform(data['overview'].fillna(''))
        
        # Réduire la dimensionnalité avec LSA
        lsa_model = TruncatedSVD(n_components=100)
        lsa_matrix = lsa_model.fit_transform(tfidf_matrix)
        
        input_vector = tfidf_vectorizer.transform([input_description])
        input_vector = lsa_model.transform(input_vector)
    elif embedding_method == 'lda':
        # Créer une matrice de documents-termes avec CountVectorizer
        count_vectorizer = CountVectorizer(stop_words='english')
        dtm = count_vectorizer.fit_transform(data['overview'].fillna(''))
        
        # Modèle de topic modeling LDA
        lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
        lda_model.fit(dtm)
        
        input_vector = count_vectorizer.transform([input_description])
        input_topic_dist = lda_model.transform(input_vector)
        input_topic_dist = input_topic_dist.reshape(1, -1)
    
    # Calcul de la similarité
    if similarity_method == 'cosine':
        if embedding_method in ['tfidf', 'lsa', 'lda']:
            similarity_scores = cosine_similarity(input_vector, tfidf_matrix).flatten()
        elif embedding_method in ['word2vec', 'doc2vec', 'bert']:
            if embedding_method == 'doc2vec':
                overview_vectors = np.array([doc2vec_model.docvecs[str(i)] for i in range(len(data))])
            elif embedding_method == 'word2vec':
                overview_vectors = np.array([np.mean([word2vec_model.wv[word] for word in overview.split() if word in word2vec_model.wv], axis=0) for overview in data['overview']])
            elif embedding_method == 'bert':
                overview_vectors = bert_model.encode(data['overview'].fillna("").tolist())
            similarity_scores = cosine_similarity([input_vector], overview_vectors)[0]
    elif similarity_method == 'jaccard':
        if embedding_method == 'tfidf':
            input_set = set(input_description.split())
            similarity_scores = [jaccard_similarity(input_set, set(overview.split())) for overview in data['overview']]
        else:
            # Pour les autres embeddings, nous utiliserons la similarité cosinus sur les ensembles de mots
            input_set = set(input_description.split())
            similarity_scores = [cosine_similarity([input_set], [set(overview.split())])[0][0] for overview in data['overview']]
    elif similarity_method == 'euclidean':
        if embedding_method == 'tfidf':
            similarity_scores = euclidean_distances(input_vector, tfidf_matrix)
        else:
            # Calcul de la distance euclidienne entre les vecteurs
            similarity_scores = [euclidean_distance(input_vector, overview_vector) for overview_vector in overview_vectors]
    elif similarity_method == 'manhattan':
        if embedding_method == 'tfidf':
            # Nous ne pouvons pas utiliser la distance de Manhattan avec TF-IDF, car cela nécessite des vecteurs numériques
            pass
        else:
            # Calcul de la distance de Manhattan entre les vecteurs
            similarity_scores = [cityblock(input_vector, overview_vector) for overview_vector in overview_vectors]
    elif similarity_method == 'pearson':
        if embedding_method == 'tfidf':
            # Nous ne pouvons pas utiliser la corrélation de Pearson avec TF-IDF, car cela nécessite des vecteurs numériques
            pass
        else:
            # Calcul de la corrélation de Pearson entre les vecteurs
            similarity_scores = [pearsonr(input_vector, overview_vector)[0] for overview_vector in overview_vectors]
    elif similarity_method == 'minkowski':
        if embedding_method == 'tfidf':
            # Nous ne pouvons pas utiliser la distance de Minkowski avec TF-IDF, car cela nécessite des vecteurs numériques
            pass
        else:
            # Calcul de la distance de Minkowski entre les vecteurs
            similarity_scores = [minkowski(input_vector, overview_vector, p=2) for overview_vector in overview_vectors]
    elif similarity_method == 'hamming':
        if embedding_method == 'tfidf':
            # Nous ne pouvons pas utiliser la distance de Hamming avec TF-IDF, car cela nécessite des vecteurs binaires
            pass
        else:
            # Calcul de la distance de Hamming entre les vecteurs
            similarity_scores = [hamming(input_vector, overview_vector) for overview_vector in overview_vectors]
    # Ajoutez d'autres méthodes de similarité au besoin
    
    # Trie les films par similarité et récupère les indices des films recommandés
    recommended_movie_indices = similarity_scores.argsort()[-n:][::-1]
    
    # Récupère les détails des films recommandés
    recommended_movies = data.iloc[recommended_movie_indices]
    
    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]
    
    # Trie les films par similarité et récupère les indices des films recommandés
    recommended_movie_indices = similarity_scores.argsort()[-n:][::-1]
    
    # Récupère les détails des films recommandés
    recommended_movies = data.iloc[recommended_movie_indices]
    
    return recommended_movies[['original_title', 'genres', 'release_date', 'overview']]

# Exemple d'utilisation
user_input = "A movie like the batman series. Something dark but fun with heroes"
recommendations = recommend_movies(user_input, embedding_method='bert', similarity_method='cosine')
display(recommendations)


,original_title,genres,release_date,overview
22956,The Lego Batman Movie,"['Action', 'Animation', 'Comedy', 'Family', 'F...",2017-02-08,In the irreverent spirit of fun that made “The...
21335,Angel City,[],1977-10-18,A detective fiction mixed with an essay-docume...
21860,Ciuleandra,"['Drama', 'Romance']",1985-01-14,The Director managed to captures perfectly the...
24298,Robin Hood,"['Drama', 'Action', 'Romance']",1991-05-13,"Yet another version of the classic epic, with ..."
18027,My Big Fat Independent Movie,['Comedy'],2005-03-04,"This film is a spoof along the lines of ""Scary..."
4901,A Shot in the Dark,"['Comedy', 'Crime']",1964-06-23,A Shot in the Dark is the second and more succ...
19646,Baji,['Action'],2015-02-06,It is an action-adventure-romance laced superh...
16499,Within the Woods,['Horror'],1978-11-23,The low budget film starring the young Bruce C...
12306,Campfire Tales,['Horror'],1997-05-16,"Anthology of famous, scary urban legends done ..."
20437,Amerikana,[],2001-05-15,Following the Dogma rules does not make this f...
